In [1]:
#モデルを自動リロード
%load_ext autoreload
%autoreload 2

from LLMSearch.ServerEmbedding import ServerEmbedding
import json


In [2]:
#embedder=ServerEmbedding()

In [3]:

vec_dict={}

In [4]:
import os
os.environ['RWKV_JIT_ON'] = '0'
os.environ["RWKV_CUDA_ON"] = '1'

from LLMSearch.Embedding.VicunaEmbedding import VicunaEmbedding
from LLMSearch.Embedding.RWKVEmbedding import RWKVEmbedding
vicuna=RWKVEmbedding(model_path="/media/kh/python/2023/RWKV-4b-Pile-3B-20230228-7963.pth",
                #strategy="",    
                strategy="cuda fp16"
                     )
#vicuna=VicunaEmbedding()

Using /home/kh/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/kh/.cache/torch_extensions/py310_cu117/wkv_cuda/build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module wkv_cuda...


ninja: no work to do.
RWKV_JIT_ON 0 RWKV_CUDA_ON 1 RESCALE_LAYER 6

Loading /media/kh/python/2023/RWKV-4b-Pile-3B-20230228-7963.pth ...
Strategy: (total 32+1=33 layers)
* cuda [float16, float16], store 33 layers
0-cuda-float16-float16 1-cuda-float16-float16 2-cuda-float16-float16 3-cuda-float16-float16 4-cuda-float16-float16 5-cuda-float16-float16 6-cuda-float16-float16 7-cuda-float16-float16 8-cuda-float16-float16 9-cuda-float16-float16 10-cuda-float16-float16 11-cuda-float16-float16 12-cuda-float16-float16 13-cuda-float16-float16 14-cuda-float16-float16 15-cuda-float16-float16 16-cuda-float16-float16 17-cuda-float16-float16 18-cuda-float16-float16 19-cuda-float16-float16 20-cuda-float16-float16 21-cuda-float16-float16 22-cuda-float16-float16 23-cuda-float16-float16 24-cuda-float16-float16 25-cuda-float16-float16 26-cuda-float16-float16 27-cuda-float16-float16 28-cuda-float16-float16 29-cuda-float16-float16 30-cuda-float16-float16 31-cuda-float16-float16 32-cuda-float16-float16 
emb.w

In [5]:
import numpy as np
txt="better man"
self=vicuna
inp_vec=self.pipeline.encode(txt)
out, state = self.model.forward(inp_vec, None)
np_state=[i.detach().cpu().numpy() for i in state]

#nanとなっているnp_stateを削除
np_state=[i for i in np_state if not np.isnan(i).any()]

np_state=[i for i in np_state if np.linalg.norm(i, axis=0, keepdims=True)<10**2]
print(len(np_state))
np_state=np.mean(np_state,axis=0)
np_state

#norm = np.linalg.norm(np_state, axis=0, keepdims=True)

#np_state/norm
#np_state=np.mean(np_state,axis=0)


22


/home/kh/anaconda3/envs/lora/lib/python3.10/site-packages/numpy/linalg/linalg.py:2541: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real


array([ 0.2308733 ,  0.27107906,  0.3374898 , ...,  0.36548182,
       -0.20509568,  0.0929408 ], dtype=float32)

In [6]:
vicuna.normalize=True
v=vicuna("better man")
v

array([[ 0.01252387,  0.01470486,  0.01830735, ...,  0.0198258 ,
        -0.01112555,  0.00504163]], dtype=float32)

In [7]:
embedder=vicuna

In [8]:
import numpy as np
norm = np.linalg.norm(v, axis=1, keepdims=True)

(v/norm).shape

(1, 2560)

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
def encode(text):
    #v=calc_vec(text,embedder.model,embedder.tokenizer)
    v=embedder(text)
    v=v/np.linalg.norm(v, axis=1, keepdims=True)
    return v


In [10]:
text_list=[
"吾輩は猫である",
"汝の名は女だ",
"I am a cat",
"ピペットで試薬を三回分取した",
"アセトンは有機溶媒である",
"非プロトン性溶媒として､THF､トルエンなどが挙げられる",
"トルエンをこぼしてしまいました",
"私は猫です",
"私は犬です",
"日本的霊性は，鎌倉時代に禅と浄土系思想によって初めて明白に顕現し，その霊性的自覚が現在に及ぶと述べる．",
]

text_vec_list=[encode(text) for text in text_list]

/home/kh/anaconda3/envs/lora/lib/python3.10/site-packages/numpy/linalg/linalg.py:2541: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real


In [11]:
target_text="猫は嫌いです"
target_text="猫の名前は?"
target_text="what is the cat name"
target_vec=encode(target_text)

In [12]:
sim_list=[]
for text,vec in zip(text_list,text_vec_list):
    #target_vec=target_vec.reshape(1,-1)
    #vec=vec.reshape(1,-1)
    sim_list.append((text,cosine_similarity(target_vec,vec)))

#コサイン類似度でソート
sim_list.sort(key=lambda x:x[1],reverse=True)
for text,sim in sim_list:
    print(text,sim)

I am a cat [[0.71107215]]
私は犬です [[0.6684967]]
私は猫です [[0.6530493]]
汝の名は女だ [[0.5569266]]
トルエンをこぼしてしまいました [[0.5476856]]
非プロトン性溶媒として､THF､トルエンなどが挙げられる [[0.5356035]]
アセトンは有機溶媒である [[0.5138507]]
ピペットで試薬を三回分取した [[0.51057684]]
吾輩は猫である [[0.5099911]]
日本的霊性は，鎌倉時代に禅と浄土系思想によって初めて明白に顕現し，その霊性的自覚が現在に及ぶと述べる． [[0.50985444]]


In [13]:
from LLMSearch.Siamese.DictEmbedder import DictEmbedder
from tqdm import tqdm
dict_embedder=DictEmbedder("develop/databricks-dolly-15k-ja-vec.bin",embedder,init=False)

bricks_dict=json.load(open("develop/databricks-dolly-15k-ja.json"))
#vec_dict_path="develop/databricks-dolly-15k-ja-vec.bin"

for brick in tqdm(bricks_dict):
    instruction=brick["instruction"]
    output=brick["output"]
    v=dict_embedder(instruction)
    v=dict_embedder(output)
    #print(instruction)
    #try:
    #    if instruction not in vec_dict:
    #        vec_dict[instruction]=embedder(instruction)
    #    if output not in vec_dict:
    #        vec_dict[output]=embedder(output)
    #except:
    #    pass


  0%|          | 0/15015 [00:00<?, ?it/s]/home/kh/anaconda3/envs/lora/lib/python3.10/site-packages/numpy/linalg/linalg.py:2541: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
  3%|▎         | 476/15015 [00:42<30:16,  8.00it/s]